In [1]:
# ------------------------------------------------------------------------------------
# markdown_to_docx.py : конвертація багатих Markdown‑документів (таблиці + формули)
#                       у формат DOCX з нативними Word‑рівняннями та стилями
# ------------------------------------------------------------------------------------
import subprocess
import sys
import importlib.util
from pathlib import Path
import textwrap

# 1. Перевіряємо наявність pypandoc і Pandoc ─────────────────────────────────────────
def ensure_dependency():
    if importlib.util.find_spec("pypandoc") is None:
        print("⏳ Встановлюємо pypandoc…")
        subprocess.check_call([sys.executable, "-m", "pip", "install", "--quiet", "pypandoc"])
    import pypandoc
    try:
        pypandoc.get_pandoc_version()
    except OSError:
        print("⏳ Pandoc не знайдено. Завантажуємо бінарну версію…")
        pypandoc.download_pandoc()   # завантажує та кешує у ~/.pandoc
    return pypandoc

pypandoc = ensure_dependency()

# 2. Markdown‑текст (можна витягати з файлу; тут тримаємо в змінні) ───────────────────
MARKDOWN_TEXT = textwrap.dedent(r"""
Запропоновано формальний, але інтуїтивно зрозумілий алгоритм, який автоматично формує дерево рівнів / класів, спираючись на (і) відстані між векторними представленнями класів, (іі) величину плутанини між ними та (ііі) прості порогові правила. Алгоритм узгоджує емпіричні спостереження авторів щодо трирівневої схеми «H / V → T / B / O → G / M / Z»  й узагальнює їх до автоматично масштабованої процедури.

---

# Запропонований метод

**Вхідні дані**

| Позначення                         | Розмірність                                                  | Пояснення                                                             |
| ---------------------------------- | ------------------------------------------------------------ | --------------------------------------------------------------------- |
| $D=\{(x_i,y_i)\}_{i=1}^{N}$        | $N\times$(зображення + мітка)                                | Навчальний набір кадрів БПЛА з їхніми початковими мітками             |
| $L=\{l_1,\dots ,l_m\}$             | $m$                                                          | **Базові класи** (тобто найдрібніші, що вже не діляться)              |
| $m$                                | скаляр                                                       | Кількість базових класів                                              |
| $B$                                | функція $\mathbb{R}^{H\times W\times3}\!\to\!\mathbb{R}^{d}$ | Back-bone, який перетворює ROI у вектор ознак розмірності $d$         |
| $λ\in[0,1]$                        | скаляр                                                       | Вага між «просторовою відстанню» й «частотою помилкових сплутувань»   |
| $δ_{\min}$                         | скаляр                                                       | Мінімальна відстань між кластерами, за якої їх ще розділяємо          |
| $ε_{\max}$                         | скаляр                                                       | Максимально допустима частка помилкових сплутувань усередині кластера |
| $Θ=(\text{FPS}_{\min},\,T_{\max})$ | вектор $2\times1$                                            | Мінімальна швидкість (кадрів/с) та максимальна латентність (мс)       |

---

## Крок 1: Витяг ознак для усіх ROI

1.1 Для кожного зображення $x_i$ виконуємо первинне детектування (будь‑який SOTA‑детектор) та отримуємо множину ROI $R_i=\{r_{i1},\dots ,r_{ik}\}$.

1.2 Для кожного ROI обчислюємо вектор ознак

$$
f_{ij}=B(r_{ij})\in\mathbb{R}^{d}.
$$

---

## Крок 2: Побудова «профілю» класу

2.1 Для кожного атомарного класу $l\in L$ збираємо множину ознак $F_l=\{f_{ij}\mid y_{ij}=l\}$.

2.2 Обчислюємо **центроїд**

$$
\mu_l=\frac{1}{|F_l|}\sum_{f\in F_l}f.
\tag{1}
$$

2.3 Одноразово навчаємо одноетапний класифікатор без ієрархії $C^{\text{flat}}$ (наприклад, простий MLP) на всіх $f_{ij}$ та формуємо **матрицю плутанини**

$$
\Omega\in[0,1]^{m\times m},\quad
\Omega_{pq}=\frac{\text{FP}(p\to q)}{\sum_{q'}\text{FP}(p\to q')},
\tag{2}
$$

де $\text{FP}(p\to q)$ — кількість неправильних присвоєнь $p\mapsto q$.

Тут під одноетапним класифікатором без ієрархії $C^{\text{flat}}$ розуміємо модель, що одразу прогнозує один з усіх $m$ базових класів без ієрархії.

---

## Крок 3: Визначення міри близькості між класами

Комбінуємо геометричну й емпіричну інформацію:

$$
M_{pq}=λ\cdot\|\mu_p-\mu_q\|_2+
(1-λ)\cdot(1-\Omega_{pq}-\Omega_{qp}),
\qquad p\neq q.
\tag{3}
$$

> *Чим більшим є $M_{pq}$ — тим легше класи розділити.*

---

## Крок 4: Ієрархічне агломеративне групування

4.1 Будуємо повнозважений граф з вагами $M_{pq}$ та застосовуємо **agglomerative clustering (average linkage)**, отримуючи дендрограму $T$.

4.2 Переходимо зверху донизу: для кожної внутрішньої вершини $S\subseteq L$ перевіряємо:

$$
\min_{p\neq q\in S}M_{pq} < δ_{\min }\;\lor\;
\frac{\sum_{p,q\in S,\,p\neq q}\Omega_{pq}}{|S|(|S|-1)} > ε_{\max }.
\tag{4}
$$

* Якщо **умова виконується** → **не розділяємо** (класи лишаються у цьому ж листі).
* Якщо **умова не виконується** → **ділимо** вершину на її дві дочірні кластери й повторюємо перевірку.

У результаті маємо $K$ рівнів $\{L_1,\dots ,L_K\}$ з дедалі дрібнішими класами; алгоритм природно повторює логіку **H/V → T/B/O → G/M/Z** з рукопису , але без ручного втручання.

---

## Крок 5. Призначення моделей на кожен рівень ієрархії

> **Мета:** для кожного рівня $k$ вибрати пару **детектор $D_k$** і **класифікатор $C_k$** так, щоб:
>
> 1. виконувалися ресурсні обмеження $Θ=(\text{FPS}_{\min},\,T_{\max})$;
> 2. модель залишалася придатною до малого/великого числа класів та розміру об’єктів.

### 5.1  Вхідні параметри (усі легко вимірювані)

| Змінна           | Розмірність / Тип  | Як отримати                                                                        | Що означає                                                                  |                                                                    |                                                       |
| ---------------- | ------------------ | ---------------------------------------------------------------------------------- | --------------------------------------------------------------------------- | ------------------------------------------------------------------ | ----------------------------------------------------- |
| (                | L\_k               | )                                                                                  | скаляр (ціле)                                                               | Злічити кількість унікальних підкласів, що залишилися на рівні $k$ | Скільки категорій треба розрізняти на поточному рівні |
| $\bar{s}_k$      | скаляр (пікселі)   | Взяти медіану значень $\sqrt{w_{ij}h_{ij}}$ для всіх ROI рівня $k$                 | Середній лінійний розмір об’єктів, які обробляє рівень                      |                                                                    |                                                       |
| $FPS_{\min}$     | скаляр (кадри / с) | Задано технічним завданням                                                         | Мінімально допустима швидкість системи в робочому режимі                    |                                                                    |                                                       |
| $T_{\max}$       | скаляр (мс)        | Задано технічним завданням                                                         | Гранична сумарна латентність повного каскаду                                |                                                                    |                                                       |
| $C_{\text{GPU}}$ | скаляр (TFLOPS)    | Паспортна потужність доступного GPU (або фактична, виміряна утилітою `nvidia-smi`) | Обчислювальний ресурс, на який спирається вибір «важкої» чи «легкої» моделі |                                                                    |                                                       |

### 5.2 Дерево рішень (детермінований алгоритм)

python
if |L_k| ≤ 2 and \bar{s}_k ≥ 48:
    D_k = YOLOv11-l
    C_k = MLP (1×256)
elif 3 ≤ |L_k| ≤ 6 and \bar{s}_k ≥ 32:
    D_k = YOLOv11-m
    C_k = FT-Transformer (d=512, h=8, 2 blocks)
elif |L_k| > 6 or \bar{s}_k < 32:
    D_k = RT-DETR-Tiny
    C_k = FT-Transformer (d=768, h=12, 4 blocks)
# ресурсна деградація
if C_GPU < 5:
    D_k = YOLO-Nano
    C_k = MobileNet-MLP

### 5.3 Правила корекції

1. **Перевірка латентності:** якщо сумарна латентність рівня $T_k > T_{\max}/K$, знижуємо варіант на один рівень у дереві.
2. **Малі об’єкти** $(\bar{s}_k < 16$ px): додаємо попередню SR-підвищувальну мережу (наприклад, ESRGAN ×2) перед $D_k$.
3. **FPS-тест:** якщо фактичний FPS $<\text{FPS}_{\min}$, замінюємо $C_k$ на компактний MLP і повторюємо тест.

### 5.4 Відтворюваність

* **Порогові значення** (48 px, 32 px, 16 px, TFLOPS = 5) обрано на базі емпіричних меж з розд. 2.5 рукопису; у реплікації їх можна варіювати, але сам алгоритм (порівняння « > / ≤ ») залишається сталим.
* **Вхідні змінні** $|L_k|$, $\bar{s}_k$, $C_{\text{GPU}}$ вимірюються однозначно; отже для тих самих даних і «заліза» вибір моделей повториться.
* Таблиця/псевдокод вище може бути реалізована в два рядки Python (`if-elif-else`), тому pipeline легко інтегрується у CI.

**Вихід:** для кожного рівня $k$ повертаємо пару $(D_k,\;C_k)$, гарантувавши, що вся ієрархія задовольняє $Θ$ і не потребує ручного тюнінгу.

---

## Крок 6: Послідовне (ззовні → всередину) навчання

6.1 Навчаємо $(D_1,F_1,C_1)$ на всьому датасеті.

6.2 Для кожного наступного рівня $k$: використовуємо тільки ті ROI, які попередній рівень відніс до надкласу, та навчаємо $(D_k,F_k,C_k)$.

6.3 Для побудови **конкатенованого вектора ознак** шукаємо підмножину шарів $\mathcal{S}$ максимізуючи

$$
\Phi(\mathcal{S})=\min_{p\neq q\in L_k}\|\mu_{p,\mathcal{S}}-\mu_{q,\mathcal{S}}\|_2,
\tag{7}
$$

аналогічно вибору у табл. 1 і рис. 5 рукопису .

---

## Крок 7: Калібрування порогів та перевірка ресурсних обмежень

7.1 Для кожного $C_k$ калібруємо поріг довіри $p_k^\star$ за критерієм Youden J.

7.2 Обчислюємо сумарну латентність і FPS; якщо не виконує $Θ$ — збільшуємо $δ_{\min }$ (отримаємо менше рівнів) та/або замінюємо моделі згідно (5)–(6).

---

## Крок 8: Експорт структури

Формуємо JSON / YAML‑файл

yaml
levels:
  - id: 1
    classes: ["H","V"]
    detector: YOLOv11-l
    classifier: FT-Transformer
    threshold: 0.82
  - id: 2
    parent_class: "V"
    classes: ["T","B","O"]
    detector: YOLOv11-m
    classifier: FT-Transformer
    threshold: 0.80
  - id: 3
    parent_class: "O"
    classes: ["G","M","Z"]
    detector: YOLOv11-s
    classifier: MLP
    threshold: 0.78

* збережені ваги моделей.

---

**Вихідні дані:**

* Дерево ієрархії $H=(L_1,\dots ,L_K)$ з моделями $\{(D_k,F_k,C_k)\}_{k=1}^{K}$.
* Конфігураційний файл для розгортання на БПЛА чи наземній станції.
* Аналітичний звіт (таблиці метрик, граф FPS‑vs‑F1, дендрограма).

---

## Опис усіх позначень

| Позначення | Розмірність                | Значення                                         |        |                                          |
| ---------- | -------------------------- | ------------------------------------------------ | ------ | ---------------------------------------- |
| $N$        | скаляр                     | Кількість зображень у датасеті                   |        |                                          |
| $x_i$      | $H\times W\times3$         | $i$-те зображення RGB                            |        |                                          |
| $y_i$      | скаляр з діапазону $[1,m]$ | Базова мітка з множини $L$                       |        |                                          |
| $m$        | скаляр                     | Число базових (найдрібніших) класів              |        |                                          |
| $f_{ij}$   | $\mathbb{R}^{d}$           | Вектор ознак $j$-го ROI на $x_i$                 |        |                                          |
| $d$        | скаляр                     | Розмірність простору ознак після $B$             |        |                                          |
| $\mu_l$    | $\mathbb{R}^{d}$           | Центроїд ознак класу $l$                         |        |                                          |
| $\Omega$   | $m\times m$                | Матриця нормованих помилок між парами класів     |        |                                          |
| $M$        | $m\times m$ трикутна       | Матриця комбінованої близькості (3)              |        |                                          |
| $K$        | скаляр                     | Кінцева кількість ієрархічних рівнів             |        |                                          |
| $L_k$      | множина                    | Підмножина класів, що розрізняється на рівні $k$ |        |                                          |
| (          | L\_k                       | )                                                | скаляр | Поточна кількість підкласів на рівні $k$ |
| $k$        | індекс $1\ldots K$         | Номер рівня в ієрархії                           |        |                                          |

## Коротке обґрунтування вигод

| Критерій                    | Ручна схема (2.4–2.5) | Автоматичний алгоритм |
| --------------------------- | --------------------- | --------------------- |
| Вибір рівнів / класів       | емпірично             | формула (4)           |
| Підбір моделей              | фіксований            | дерево рішень (5–6)   |
| Пороги довіри               | не описано            | калібрування 7.1      |
| Підтримка FPS               | декларативно          | перевірка 7.2         |
| Масштабування на нові класи | ручне                 | повторити кроки 1–4   |

Алгоритм робить побудову структури **трасованою, відтворюваною** й адаптивною до нових наборів класів чи апаратних обмежень, зберігаючи концепцію, закладену авторами .



## Крок 5. Призначення моделей кожному рівню $k$ (удосконалене дерево рішень)

Нехай

* $|L_k|$ — кількість підкласів, які треба розрізнити на рівні $k$;
* $\bar{s}_k$ — медіанний розмір ROI (у пікселях) між усіма підкласами рівня;
* $C_{\mathrm{GPU}}$ — доступна обчислювальна потужність (TFLOPS).

| Вхідні умови                               | Детектор $D_k$ | Витяг $F_k$                       | Класифікатор $C_k$                   |                              |                                               |
| ------------------------------------------ | -------------- | --------------------------------- | ------------------------------------ | ---------------------------- | --------------------------------------------- |
| (                                          | L\_k           | !\le!2) та $\bar{s}_k\!>\!48$ px  | **YOLOv11-l** (≈130 FPS на RTX 3090) | Стандартний CSPDarknet-53    | **Linear MLP** (1 прих. шар, 256 нейронів)    |
| (3!\le!                                    | L\_k           | !\le!6) & $\bar{s}_k\!\ge\!32$ px | **YOLOv11-m**                        | CSPDarknet-53 + SPP-FPN      | **FT-Transformer** (d = 512, h = 8, 2 блоки)  |
| (                                          | L\_k           | !>!6) або $\bar{s}_k\!<\!32$ px   | **RT-DETR-Tiny** (ембединг 256)      | CSPDarknet-53 + FPN + CARAFE | **FT-Transformer** (d = 768, h = 12, 4 блоки) |
| При дефіциті TFLOPS ($C_{\mathrm{GPU}}<5$) | **YOLO-Nano**  | Lightweight FPN                   | **MobileNet-MLP**                    |                              |                                               |

""").strip()

# 3. Шляхи, які використовуємо
TMP_MD   = Path("temp_input.md")
DOCX_OUT = Path("hierarchical_algorithm.docx")

# 4. Записуємо Markdown у тимчасовий файл
TMP_MD.write_text(MARKDOWN_TEXT, encoding="utf-8")

# 5. Конвертація у DOCX через Pandoc
print("🚀 Запускаємо Pandoc → DOCX…")
pypandoc.convert_file(
    str(TMP_MD),
    to="docx",
    outputfile=str(DOCX_OUT),
    extra_args=[
        "--standalone",
        "--toc",                     # автоматичний зміст (за потреби)
        # "--reference-doc=template.docx",  # розкоментуйте для власного шаблону
    ]
)

# 6. Прибирання тимчасових файлів
TMP_MD.unlink(missing_ok=True)

print(f"✅ DOCX успішно створено: {DOCX_OUT.resolve()}")


⏳ Встановлюємо pypandoc…
🚀 Запускаємо Pandoc → DOCX…


TypeError: 'WindowsPath' object is not iterable